In [ ]:
import numpy as np
import networkx as nx
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pydot
import meshplot as mp
import open3d as o3d
from networkx.drawing.nx_pydot import graphviz_layout
from itertools import combinations


In [ ]:
branches = []
borders = {}
with open( '../results/faust_54098/faust_54098.txt' ) as file:
    for idx,line in enumerate(file):
        line =line.replace('\n', '')
        contents = line.split(" ")


        if tuple(contents[1:4]) in borders:
            borders[ tuple(contents[1:4]) ].add(idx)
        else:
            borders[ tuple(contents[1:4]) ] = set([idx])

        if tuple(contents[-3:]) in borders:
            borders[ tuple(contents[-3:]) ].add(idx)
        else:
            borders[ tuple(contents[-3:]) ] = set([idx])

        branches.append( np.reshape( np.array( contents[1:]).astype(np.float32), ((len(contents) - 1) // 3, 3) ) )

len(branches)

In [ ]:
mesh = o3d.io.read_triangle_mesh('../data/humans/mesh/faust_6890.off')

p = mp.plot( np.asarray(mesh.vertices) , shading={'point_size': 0.01, 'point_color':'#AA0000' },return_plot=True)
cmap1 = cm.get_cmap('tab20b')
cmap2 = cm.get_cmap('tab20c')
for i, branch in enumerate(branches):
    cmap = cmap1 if i < 20 else cmap2
    c = '#'
    for x in (np.array(cmap( (i)/20) )[0:3] * 255).astype(np.uint32):
        h = hex(x)[2:]
        c += h if len(h) == 2 else '0' + h

    p.add_points( branch, shading={'point_size': 0.1, 'point_color':c })

In [ ]:
grafo = nx.Graph()
grafo.add_nodes_from( np.arange(0,len(branches)) )
grafo.add_edges_from( [ t for s in borders.values() for t in combinations(s, 2) ] )


In [ ]:
nx.draw(grafo, with_labels=True)

In [ ]:
nx.number_connected_components(grafo)

In [ ]:
print( len(grafo.nodes), len(grafo.edges) ) # es arbol si m = n - 1!

Quiero elegir la raiz que me de el arbol mas bajito... nose porque... porque es mas lindo nomas

In [ ]:
raiz = np.argmin( np.max( [ [ len(k) for k in d.values() ] for d in dict(nx.all_pairs_shortest_path(grafo)).values() ], axis=1 ) )
arbol = nx.bfs_tree( grafo, raiz )

In [ ]:
pos = graphviz_layout(arbol, prog="dot")
pos = { int(k): v for k,v in pos.items()}
nx.draw(arbol, pos, with_labels=True)
plt.show()

# Test grafo de centros

In [7]:
import numpy as np
import open3d as o3d
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pydot
import meshplot as mp
from networkx.drawing.nx_pydot import graphviz_layout

In [3]:
def createGraph( branches ):
    graph = nx.Graph()
    for branch in branches:
        lastNode = None
        for joint in branch:
            nodeCode = hash( (' ').join(joint) )
            
            graph.add_node( nodeCode, position=np.array(joint).astype(np.float32) )
            if lastNode != None:
                graph.add_edge( nodeCode, lastNode )

            lastNode = nodeCode

    return graph

meshFile = '../data/humans/mesh/faust_6890.off'
skeletonFile = '../data/humans/centerline/faust_6890.txt'

mesh = o3d.io.read_triangle_mesh( meshFile )
centers = []

with open( skeletonFile ) as file:
    for idx, line in enumerate(file):
        line =line.replace('\n', '')
        contents = line.split(" ")
        centers.append( [ contents[i:i+3] for i in range(1,len(contents),3)] )

graph = createGraph( centers )

print( len(graph.nodes), len(graph.edges))
print( nx.number_connected_components(graph))

## Conexo y m = n-1 => arbol

[['0.407979', '-0.403275', '0.0798797'], ['0.406916', '-0.398029', '0.0651192'], ['0.407569', '-0.401849', '0.0540092'], ['0.407914', '-0.411452', '0.0462927'], ['0.407984', '-0.415631', '0.044058'], ['0.407982', '-0.421294', '0.0416138'], ['0.407844', '-0.432957', '0.038328'], ['0.407726', '-0.439744', '0.0367879'], ['0.407684', '-0.441706', '0.0363928']]
407 406
1


In [ ]:
for node in graph.nodes:
    assert hash( node['position']
